In [ ]:
pip install numpy pandas matplotlib pillow opencv-python


In [ ]:
import os
import matplotlib.pyplot as plt
import cv2


In [ ]:
os.getcwd()

In [ ]:
os.chdir('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign')

In [ ]:
os.getcwd()

In [ ]:
path='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign'
images=os.listdir(path)
len(images)

path='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant'
images=os.listdir(path)
len(images)


In [ ]:
pip install torch torchvision timm

In [ ]:
path_benign='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign'
path_malignant='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant'


def load_images_with_labels(path,label):
    images=os.listdir(path)
    labeled_images=[]
    for img in images:
        img_path=os.path.join(path,img)
        img_arr=cv2.imread(img_path)
        if img_arr is not None:
            labeled_images.append((img_arr,label))
    return labeled_images       

benign_images=load_images_with_labels(path_benign,0)
malignant_images=load_images_with_labels(path_malignant,1)

data_images=benign_images+malignant_images

print(len(data_images))


In [ ]:
import random
path_benign='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign'
path_malignant='C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant'


def load_images_with_labels(path,label,num_samples):
    images=os.listdir(path)
    if num_samples>len(images):
        num_samples=len(images)
    selected_images=random.sample(images,num_samples)    
    labeled_images=[]
    for img in selected_images:
        img_path=os.path.join(path,img)
        img_arr=cv2.imread(img_path)
        if img_arr is not None:
            labeled_images.append((img,label))
    return labeled_images       

benign_images=load_images_with_labels(path_benign,0,1000)
malignant_images=load_images_with_labels(path_malignant,1,1000)

data_images_mod=benign_images+malignant_images

print(len(data_images_mod))

In [ ]:
import random

random.shuffle(data_images_mod)

for img,label in data_images_mod:
    print(label,img)

In [ ]:
pip install pandas openpyxl

In [ ]:
os.chdir('C:\\ML_PROJECTS')
os.getcwd()

In [ ]:
import pandas as pd
data = {
    'image_name': [img_name for img_name,_ in data_images_mod],
    'status': [label for _, label in data_images_mod]
}

df = pd.DataFrame(data)


excel_path = 'melanoma_org_1.xlsx'
df.to_excel(excel_path, index=False)

print(f"Excel file saved to {excel_path}")

In [ ]:
pip install tensorflow

In [ ]:
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import psutil

memory_info = psutil.virtual_memory()
print(f"Total memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available memory: {memory_info.available / (1024 ** 3):.2f} GB")


In [ ]:
import gc
gc.collect()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org_1.xlsx')


base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
   
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
 
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status==0:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign', img_name)
        else:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
 
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'vgg16_features_batch_{i//batch_size}.pkl')
    # Clear memory
    gc.collect()


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org_1.xlsx')


base_model = VGG19(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
   
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
   
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status==0:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign', img_name)
        else:
          img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
 
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'vgg19_features_batch_{i//batch_size}.pkl')
   
    gc.collect()




In [ ]:
os.chdir('C:\\ML_PROJECTS')


In [ ]:
import pickle
import numpy as np
data_list=[]

for i in range(10):
    filename=f"vgg16_features_batch_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)

concatenated_data = np.concatenate(data_list, axis=0)

with open('vgg16_features_concatenated.pkl', 'wb') as f:
    pickle.dump(concatenated_data, f)

print("Concatenation complete and saved to 'vgg16_features_concatenated.pkl'")

In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(10):
    filename=f"vgg19_features_batch_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('vgg19_features_concatenated.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'vgg19_features_concatenated.pkl'")


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org_1.xlsx')


base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
    
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features


batch_size = 200
all_features = []


for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
  
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'densenet_features_batch_{i//batch_size}.pkl')
    
    gc.collect()

print("Feature extraction complete and saved.")


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(10):
    filename=f"densenet_features_batch_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('densenet_features_concatenated.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'densenet_features_concatenated.pkl'")


In [ ]:
import os 
os.getcwd()

In [ ]:
os.chdir("C:\\ML_PROJECTS")

In [ ]:
import pickle

f1="vgg16_features_concatenated.pkl"
f2="vgg19_features_concatenated.pkl"
f3="densenet_features_concatenated.pkl"

with open(f1,'rb') as f:
    data=pickle.load(f)

for img,f_v in data:
    print(f'{img},{f_v.shape}')

In [ ]:
import numpy as np

input_shape=12288

flattened_vector=np.random.rand(input_shape)


num_samples=1

height=64

width=64

no_of_channels=3

new_feature_vector=flattened_vector.reshape(num_samples,height,width,no_of_channels)



In [ ]:
import tensorflow as tf

from tensorflow.keras import layers, models


model = models.Sequential()


model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))  


model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())


model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


In [ ]:
import os
os.getcwd()
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org_1.xlsx')


base_model = InceptionV3(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features

batch_size = 200
all_features = []

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
    
    
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'inceptionv3_features_batch_{i//batch_size}.pkl')
    
    
    gc.collect()


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(10):
    filename=f"inceptionv3_features_batch_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('inceptionv3_features_concatenated.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'inceptionv3_features_concatenated.pkl'")


In [ ]:
import pickle
import numpy as np

with open('inceptionv3_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import gc


df = pd.read_excel('melanoma_org_1.xlsx')


base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

def process_batch(batch_df):
    features = []
    for index, row in batch_df.iterrows():
        img_name = row['image_name']
        status = row['status']
        if status == 0:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\benign', img_name)
        else:
            img_path = os.path.join('C:\\ML_PROJECTS\\melanoma_cancer_dataset\\train\\malignant', img_name)
        print(f"Processing image: {img_path}")
        try:
            img = load_and_preprocess_image(img_path)
            feature_vector = model.predict(img)
            feature_vector = feature_vector.flatten()
            features.append([img_name, feature_vector])
        except FileNotFoundError as e:
            print(f"File not found: {img_path}")
            continue
    return features

batch_size = 200
all_features = []

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_features = process_batch(batch_df)
    all_features.extend(batch_features)
    
    
    intermediate_df = pd.DataFrame(batch_features, columns=['Image_name', 'Feature_vector'])
    intermediate_df.to_pickle(f'resnet50_features_batch_{i//batch_size}.pkl')
    
    
    gc.collect()


In [ ]:
import pickle
import numpy as np

data_list=[]

for i in range(10):
    filename=f"resnet50_features_batch_{i}.pkl"
    with open(filename,'rb') as f:
        data=pickle.load(f)
        data_list.append(data)


concatenated_data=np.concatenate(data_list,axis=0)

with open('resnet50_features_concatenated.pkl','wb') as f:
    pickle.dump(concatenated_data,f)

print("Concatenation complete and saved to 'resnet50_features_concatenated.pkl'")


In [ ]:
import pickle
import numpy as np

with open('resnet50_features_concatenated.pkl','rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'image:{images},{f_v.shape}')

In [ ]:
import os
os.getcwd()
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import pickle
import numpy as np

f1='vgg16_features_concatenated_test.pkl'
f2='vgg19_features_concatenated.pkl'
f3='resnet50_features_concatenated.pkl'
f4='densenet_features_concatenated.pkl'
# f5='inceptionv3_features_concatenated.pkl'

with open(f4,'rb') as f:
    data=pickle.load(f)

for images,f_v in data:
    print(f'{images}...{f_v.shape}')


In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers, models

original_array=np.random.rand(100352)

extended_array=np.pad(original_array,(0,115),mode='constant')

num_samples = 1
height = 183
width = 183
no_of_channels = 3
reshaped_array = extended_array.reshape(num_samples, height, width, no_of_channels)

print(reshaped_array.shape)


model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(182,182,3)))  
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())


model.add(layers.Dense(1024, activation='relu'))



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers, models

original_array=np.random.rand(131072)

extended_array=np.pad(original_array,(0,1228),mode='constant')

num_samples = 1
height = 210
width = 210
no_of_channels = 3
reshaped_array = extended_array.reshape(num_samples, height, width, no_of_channels)

print(reshaped_array.shape)


model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(182,182,3)))  
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))


model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())


model.add(layers.Dense(1024, activation='relu'))



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

pred_fv=model.predict(f_v)


model.summary()


In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(92,92,3)))  
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(1024, activation='relu'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='vgg16_features_concatenated.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,304),mode='constant')
    num_samples=1
    height=92
    width=92
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model1.predict(reshaped_array)
    filename=f'vgg16_{image}_reduced_features.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model1 = models.Sequential()
model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(92,92,3)))  
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(1024, activation='relu'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='vgg16_features_concatenated.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,304),mode='constant')
    num_samples=1
    height=92
    width=92
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model1.predict(reshaped_array)
    filename=f'vgg19_{image}_reduced_features.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import os
images=os.listdir('C:\\ML_PROJECTS')

count=0

for image in images:
    if image.startswith ('vgg16_') and image.endswith('_reduced_features.pkl'):
        count+=1

print(count)


In [ ]:
import os
images=os.listdir('C:\\ML_PROJECTS')

count=0

for image in images:
    if image.startswith ('vgg19_') and image.endswith('_reduced_features.pkl'):
        count+=1

print(count)


In [ ]:
import os
images=os.listdir('C:\\ML_PROJECTS')


for image in images:
    if image.startswith ('vgg19_') and image.endswith('_reduced_features.pkl'):
        with open(image,'rb') as f:
            feature_vector=pickle.load(f)
            print((feature_vector.shape))
        

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(210,210,3)))  
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(1024, activation='relu'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='inceptionv3_features_concatenated.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    # print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,1228),mode='constant')
    num_samples=1
    height=210
    width=210
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model2.predict(reshaped_array)
    filename=f'inceptionv3_{image}_reduced_features.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import os
images=os.listdir('C:\\ML_PROJECTS')

count=0

for image in images:
    if image.startswith ('inceptionv3_') and image.endswith('_reduced_features.pkl'):
        count+=1

print(count)


In [ ]:
import pickle
import os

images=os.listdir('C:\\ML_PROJECTS')

for image in images:
    if image.startswith('inceptionv3_') and image.endswith('_reduced_features.pkl'):
        with open(image,'rb') as f:
            feature_vector=pickle.load(f)
            print(feature_vector.shape[1])

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np

model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(182,183,3)))  
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Flatten())
model3.add(layers.Dense(1024, activation='relu'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

f1='resnet50_features_concatenated.pkl'
with open(f1,'rb') as f:
    data=pickle.load(f)

for image,f_v in data:
    input_shape=f_v.shape
    #print(input_shape[0])
    original_array=np.random.rand(input_shape[0])
    padded_array=np.pad(original_array,(0,115),mode='constant')
    num_samples=1
    height=183
    width=183
    no_of_channels=3
    reshaped_array = padded_array.reshape(num_samples, height, width, no_of_channels)
    features=model3.predict(reshaped_array)
    filename=f'resnet50_{image}_reduced_features.pkl'

    with open(filename,'wb') as f:
       pickle.dump(features,f)
    
    print(f'Dimension reduced and features saved for {image} successfully to {filename}!!')
    
model.summary()

In [ ]:
import os
images=os.listdir('C:\\ML_PROJECTS')

count=0

for image in images:
    if image.startswith ('resnet50_') and image.endswith('_reduced_features.pkl'):
        count+=1

print(count)


In [ ]:
import pickle
import os

images=os.listdir('C:\\ML_PROJECTS')

for image in images:
    if image.startswith('resnet50_') and image.endswith('_reduced_features.pkl'):
        with open(image,'rb') as f:
            feature_vector=pickle.load(f)
            print(feature_vector)

In [ ]:
import os

os.getcwd()

os.chdir('C:\\ML_PROJECTS')

In [ ]:
import os
import numpy as np
import pickle
images=os.listdir('C:\\ML_PROJECTS')
count=0

for image in images:
    if image.startswith(f'{model}_') and image.endswith('_reduced_features.pkl'):
        count+=1
        i_name=image
        if count==1:
            break

print(i_name)

parts=i_name.split('_')

print(parts)

new_i_name=parts[1]+'_'+parts[2]

print(new_i_name)

f1="densenet_features_concatenated.pkl"

feature=[]

for model in ['vgg16','vgg19','inceptionv3','resnet50']:
    filename=f'{model}_{new_i_name}_reduced_features.pkl'
    with open(filename,'rb') as f:
        f_v=pickle.load(f)
        print(f_v.shape[1])
        feature.append(f_v)
with open(f1,'rb') as f2:
    data=pickle.load(f2)
    for images,f_v_1 in data:
        if new_i_name in images:
            print(f'{f_v_1.shape[0]}')
            feature.append(f_v_1)

print(len(feature))
print(feature)

# feature_arr=np.array(feature)

# print(feature_arr.shape)
    

In [ ]:
import os
import numpy as np
import pickle


images = os.listdir('C:\\ML_PROJECTS')


f1 = "densenet_features_concatenated.pkl"


models = ['vgg16', 'vgg19', 'inceptionv3', 'resnet50']


for image in images:
   
    if image.startswith('vgg16_') and image.endswith('_reduced_features.pkl'):
        parts = image.split('_')
        new_i_name = parts[1] + '_' + parts[2]

        feature = []

       
        for model in models:
            filename = f'{model}_{new_i_name}_reduced_features.pkl'
            try:
                with open(filename, 'rb') as f:
                    f_v = pickle.load(f)
                    print(f'Loaded {filename}: {f_v.shape[1]}')
                    feature.append(f_v)
            except FileNotFoundError:
                print(f'File {filename} not found. Skipping this model.')

        
        with open(f1, 'rb') as f2:
            data = pickle.load(f2)
            for images, f_v_1 in data:
                if new_i_name in images:
                    print(f'Appending features from concatenated file: {f_v_1.shape[0]}')
                    feature.append(f_v_1)

        
        output_filename = f'concatenated_features_{new_i_name}.pkl'
        with open(output_filename, 'wb') as f_out:
            pickle.dump(feature, f_out)
            print(f'Saved concatenated features to {output_filename}')


In [ ]:
import os

images=os.listdir('C:\\ML_PROJECTS')
count=0
for image in images:
  if image.startswith('concatenated_features_') and image.endswith('.pkl'):
      count+=1

print(count)

In [ ]:
import os
import pickle
import pandas as pd

df=pd.read_excel('melanoma_org_1.xlsx')

i_name=df['image_name'].values
label=df['status'].values

images=os.listdir('C:\\ML_PROJECTS')
for image in images:
    if image.startswith('concatenated_features_') and image.endswith('.pkl'):
        parts=image.split('_')
        # print(parts)
        name_parts=parts[3].split('.')
        # print(name_parts)
        image_name=parts[2]+'_'+name_parts[0]+'.'+name_parts[1]
        print(image_name)
        print('\t')
        with open(image,'rb') as f:
            data=pickle.load(f)
            for f_v in data:
                print(f_v)
        if image_name in i_name:
            index=list(i_name).index(image_name)
            image_label=label[index]
            print(image_label)
           
        


In [ ]:
import os
import pickle
import pandas as pd


df = pd.read_excel('melanoma_org_1.xlsx')
i_name = df['image_name'].values
label = df['status'].values


data_to_save = []

images = os.listdir('C:\\ML_PROJECTS')

for image in images:
    if image.startswith('concatenated_features_') and image.endswith('.pkl'):
        parts = image.split('_')
        name_parts = parts[3].split('.')
        image_name = parts[2] + '_' + name_parts[0] + '.' + name_parts[1]
        print(image_name)
        
        with open(os.path.join('C:\\ML_PROJECTS', image), 'rb') as f:
            data = pickle.load(f)
            concatenated_features = [f_v for f_v in data]
        
        if image_name in i_name:
            
            index = list(i_name).index(image_name)
            image_label = label[index]
            print(f'Image: {image_name}, Label: {image_label}')
            
           
            data_to_save.append([image_name, image_label, concatenated_features])
        else:
            print(f'Image name {image_name} not found in labels')


df_to_save = pd.DataFrame(data_to_save, columns=['image_name', 'label', 'feature_vector'])


df_to_save.to_excel('features_with_labels.xlsx', index=False)

print('Saved feature vectors and labels to features_with_labels.xlsx')


In [ ]:
pip install xgboost

In [ ]:
 pip install --upgrade pip

In [ ]:
import pandas as pd
import ast


df = pd.read_excel('features_with_labels.xlsx')

X=df['feature_vector'].values
y=df['label'].values

print(type(X))

def string_to_array(string):
    try:
        list_representation=eval(string)
        return np.array(list_representation)
    except:
        return np.nan 


df['feature_vector']=df['feature_vector'].apply(string_to_array)

X=df['feature_vector'].values
print(type(X))

X = np.vstack(df['feature_vector'].values)

print(type(X))





In [ ]:
import os

os.getcwd()
os.chdir('C:\\ML_PROJECTS')
os.getcwd()

In [ ]:
import numpy as np
import pandas as pd

df=pd.read_excel('melanoma_org_1.xlsx')

df.head()

status=df['status'].values
print(status)

image_name=df['image_name'].values

print(image_name)



In [ ]:
import os
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import os
import shutil

base_directory='C:\\ML_PROJECTS'

concatenated_feature_dir=os.path.join(base_directory,'concatenated_feature_dir')

os.makedirs(concatenated_feature_dir,exist_ok=True)

for filename in os.listdir(base_directory):
    if filename.startswith('concatenated_features_') and filename.endswith('.pkl'):
        source_path=os.path.join(base_directory,filename)
        target_path=os.path.join(concatenated_feature_dir,filename)
        shutil.move(source_path,target_path)
        print(f"Moved {filename} to {concatenated_feature_dir}")

In [ ]:
import os
import shutil
import pandas as pd


df = pd.read_excel('melanoma_org_1.xlsx')


base_directory = 'C:\\ML_PROJECTS'
source_directory = os.path.join(base_directory, 'concatenated_feature_dir')


label_0_dir = os.path.join(base_directory, 'label_0')
label_1_dir = os.path.join(base_directory, 'label_1')


os.makedirs(label_0_dir, exist_ok=True)
os.makedirs(label_1_dir, exist_ok=True)


for idx, row in df.iterrows():
    label = row['status']  
    image_name = row['image_name']  
    
    
    for filename in os.listdir(source_directory):
        
        if image_name in filename:
            source_path = os.path.join(source_directory, filename)
            
            
            if label == 0:
                target_path = os.path.join(label_0_dir, filename)
            elif label == 1:
                target_path = os.path.join(label_1_dir, filename)
            
            
            shutil.move(source_path, target_path)
            print(f"Moved {filename} to {target_path}")


In [ ]:
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import os
import cv2  
import numpy as np


image_data = []
labels = []

def load_images_from_folder(folder, label):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        print(f"Loading image from path: {img_path}")
        
        
        try:
            img = cv2.imread(img_path)
            if img is None:
                raise ValueError(f"Unable to read image {filename}")
            
            
            img_resized = cv2.resize(img, (64, 64))
            
            
            image_data.append(img_resized.flatten())
            labels.append(label)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")


base_directory = 'C:\\ML_PROJECTS'
label_0_dir = os.path.join(base_directory, 'label_0')
label_1_dir = os.path.join(base_directory, 'label_1')


load_images_from_folder(label_0_dir, 0)


load_images_from_folder(label_1_dir, 1)


X = np.array(image_data)
y = np.array(labels)


print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


if X.ndim == 1:
    X = X.reshape(-1, 1)  
if y.ndim > 1:
    y = y.ravel()  




In [ ]:
import os
os.chdir('C:\\ML_PROJECTS')

In [ ]:
import os

def check_file_paths(folder):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            print(f"File exists: {img_path}")
        else:
            print(f"File not found: {img_path}")


check_file_paths(label_0_dir)
check_file_paths(label_1_dir)


In [ ]:
import pickle

def verify_images(folder):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        
        try:
            with open(img_path,'rb') as f:
                file=pickle.load(f)
                print(f"Image verified: {img_path}")
        except (IOError, SyntaxError) as e:
            print(f"Corrupted image or unreadable file: {img_path}, Error: {e}")


verify_images(label_0_dir)
verify_images(label_1_dir)


In [ ]:
import os
import pandas as pd
import numpy as np

df=pd.read_excel('features_with_labels.xlsx')
count=0
for idx,row in df.iterrows():
    image_name=row['image_name']
    label=row['label']
    f_v=row['feature_vector']
    f_v_cleaned = f_v.replace('dtype=float32', '').replace('array', 'np.array')
    try:
        f_v_array = eval(f_v_cleaned)  
    except Exception as e:
        print(f"Error parsing feature vector: {e}")
        continue
    count+=1
    if count==1:
        break


print(type(f_v_array))


print(f_v_array.flatten())


    

    

In [ ]:
import os
os.getcwd()


In [ ]:
import pickle
filename='densenet_features_batch_0.pkl'
with open(filename,'rb') as f:
    file=pickle.load(f)

print(file)


In [ ]:
import pickle
import os
import numpy as np


base_directory='C:\\ML_PROJECTS'

label_0_dir=os.path.join(base_directory,"label_0")

print(label_0_dir)

image_name="concatenated_features_melanoma_13.jpg.pkl"

image_dir=os.path.join(label_0_dir,image_name)

print(image_dir)

with open(image_dir,'rb') as f:
    file=pickle.load(f)
    
 
print(file)


In [ ]:
import pickle
models=['vgg16','vgg19','inceptionv3','resnet50']

for model in models:
    filename=f'{model}_melanoma_9.jpg_reduced_features.pkl'
    with open(filename,'rb') as f:
        data=pickle.load(f)
    print(data)

In [ ]:
import pickle
import numpy as np
filename='vgg16_melanoma_9.jpg_reduced_features.pkl'
with open(filename,'rb') as f:
    data=pickle.load(f)

f2='resnet50_melanoma_9.jpg_reduced_features.pkl'
f3='vgg19_melanoma_9.jpg_reduced_features.pkl'
f4='inceptionv3_melanoma_9.jpg_reduced_features.pkl'
f5='densenet_features_concatenated.pkl'
with open(f2,'rb') as f:
    data1=pickle.load(f)

with open(f3,'rb') as f:
    data2=pickle.load(f)

with open(f4,'rb') as f:
    data3=pickle.load(f)
with open(f5,'rb') as f:
    data4=pickle.load(f)
list=[]

def get_f_v_densenet(data4):
    for img,f_v in data4:
        if 'melanoma_9_' in img:
           print(img)
    return f_v        
i=0     

f_v=get_f_v_densenet(data4)



for j in range(1024):
    x=0
    x=(data[i][j]+data1[i][j]+data2[i][j]+data3[i][j]+f_v[j])/5
    list.append(x)

print(list)

arr=np.array(list)

print(arr.shape)





In [ ]:
f5='densenet_features_concatenated.pkl'

with open(f5,'rb') as f:
    data=pickle.load(f)

def get_f_v(data):
    for img,f_v in data:
        if 'melanoma_9_' in img:
            print(img)
    return f_v

f_v=get_f_v(data)

for i in range(1023):
    print(f_v[i])

In [ ]:
import os
import pandas as pd
import numpy as np

base_dir='C:\\ML_PROJECTS'

label_0_dir=os.path.join(base_dir,'label_0')
label_1_dir=os.path.join(base_dir,'label_1')

def search_label_0_dir(filename):
    for files in os.listdir(label_0_dir):
       if filename in files:
           return 1
    return 0

def search_label_1_dir(filename):
    for files in os.listdir(label_1_dir):
       if filename in files:
           return 1
    return 0

df=pd.read_excel('melanoma_org_1.xlsx')
label=[]
for idx,row in df.iterrows():
    image_name=row['image_name']
    filename=f'concatenated_features_{image_name}.pkl'
    x=search_label_0_dir(filename)
    y=search_label_1_dir(filename)
    if x:
        label.append(0)
    if y:
        label.append(1)

print(label)

arr=np.array(label)

print(arr.shape)
    
    

In [ ]:
import pickle
import numpy as np
import os
import pandas as pd



def get_feature_vector(image_name):
    l=[]
    x=0
    i=0
    f1=f'vgg16_{image_name}_reduced_features.pkl'
    with open(f1,'rb') as f:
        data1=pickle.load(f)
    f2=f'vgg19_{image_name}_reduced_features.pkl'
    with open(f2,'rb') as f:
        data2=pickle.load(f)
    f3=f'resnet50_{image_name}_reduced_features.pkl'
    with open(f3,'rb') as f:
        data3=pickle.load(f)
    f4=f'inceptionv3_{image_name}_reduced_features.pkl'
    with open(f4,'rb') as f:
        data4=pickle.load(f)
    f5=f'densenet_features_concatenated.pkl'
    with open(f5,'rb') as f:
        data5=pickle.load(f)
    f_v=get_f_v_densenet(data5,image_name)
    for j in range(1024):
        x=data1[i][j]+data2[i][j]+data3[i][j]+data4[i][j]+f_v[j]
        l.append(x)

    print(l)
        
        
            



def get_f_v_densenet(data5,image_name):
    for img,f_v in data5:
        if image_name in img:
           return f_v       

base_dir='C:\\ML_PROJECTS'

label_0_dir=os.path.join(base_dir,'label_0')
label_1_dir=os.path.join(base_dir,'label_1')

def search_label_0_dir(filename):
    for files in os.listdir(label_0_dir):
       if filename in files:
           return 1
    return 0

def search_label_1_dir(filename):
    for files in os.listdir(label_1_dir):
       if filename in files:
           return 1
    return 0

df=pd.read_excel('melanoma_org_1.xlsx')
label=[]
for idx,row in df.iterrows():
    image_name=row['image_name']
    print(image_name)
    filename=f'concatenated_features_{image_name}.pkl'
    get_feature_vector(image_name)
    x=search_label_0_dir(filename)
    y=search_label_1_dir(filename)
    print('label:\t')
    if x:
        label.append(0)
        print(x)
    if y:
        label.append(1)
        print(y)

print(label)


    
    

